In [129]:
import pandas as pd
import numpy as np

import glob
import json
from collections import defaultdict

from tqdm.auto import tqdm

import warnings
warnings.filterwarnings('ignore')

In [3]:
jsons = glob.glob('reports/*.json')

In [4]:
with open('col_names', 'r') as f:
    cols = f.read().split('\n')

In [5]:
def get_data(jsons, cols):
    d = defaultdict(list)
    for elem in tqdm(jsons):
        spec_cols = ['ticker']
        ticker = elem[8:].split('_')[0]
        with open(elem, 'r') as f:
            elem_dict = json.load(f)['financials']
        if isinstance(elem_dict, dict):
            elem_annuals = elem_dict['annuals']
            n_size = len(elem_annuals['Fiscal Year'])
            for col in elem_dict['financial_template_parameters']:
                d[col].extend(elem_dict['financial_template_parameters'][col] * n_size)
                spec_cols.append(col)
            d['ticker'].extend([ticker] * n_size)
            for col in elem_annuals:
                if not isinstance(elem_annuals[col], dict):
                    d[col].extend(elem_annuals[col])
                    spec_cols.append(col)
                    assert len(elem_annuals[col]) == n_size
                else:
                    for col_ in elem_annuals[col]:
                        if col_ not in spec_cols:
                            d[col_].extend(elem_annuals[col][col_])
                            spec_cols.append(col_)
                            assert len(elem_annuals[col][col_]) == n_size
            for col in list(set(cols) - set(spec_cols)):
                d[col].extend([None] * n_size)
    return pd.DataFrame.from_dict(d)

In [6]:
df = get_data(jsons, cols)
print(df.shape)
df.head()

100%|██████████| 1493/1493 [00:04<00:00, 316.56it/s]


(24769, 266)


,ind_template,REITs,IsDirect,ticker,Fiscal Year,Preliminary,Revenue per Share,EBITDA per Share,EBIT per Share,Earnings per Share (Diluted),...,Net Interest Income (for Banks),Other Assets for Insurance Companies,Future Policy Benefits,Securities & Investments,Total Expenses,Unearned Income,Net Investment Income,Equity Investments,period,Money Market Investments
0,N,N,Indirect Method,TPTX,2017-12,0,0,-1.116,-1.12,-1.111,...,None,None,None,None,None,None,None,None,None,None
1,N,N,Indirect Method,TPTX,2018-12,0,0,-1.707,-1.716,-1.659,...,None,None,None,None,None,None,None,None,None,None
2,N,N,Indirect Method,TPTX,2019-12,0,0,-3.156,-3.222,-2.99,...,None,None,None,None,None,None,None,None,None,None
3,N,N,Indirect Method,TPTX,2020-12,0,0.612,-3.904,-3.962,-3.85,...,None,None,None,None,None,None,None,None,None,None
4,N,N,Indirect Method,TPTX,TTM,0,1.213,-3.298,-3.355,-3.31,...,None,None,None,None,None,None,None,None,None,None


In [7]:
def preprocess(data):
    df = data.copy()
    df.drop(df[(df['Fiscal Year'] == 'TTM') | (df['Fiscal Year'] == '')].index, inplace=True)
    df['period'] = df['Fiscal Year'].apply(lambda x: x[:4]).astype(np.int16)
    df.replace({None: np.nan, 'N/A': np.nan, '-': np.nan, '': np.nan}, inplace=True)

    for col in df.columns[df.dtypes == 'object']:
        try:
            df[col] = df[col].astype(np.int32)
            continue
        except:
            pass
        try:
            df[col] = df[col].astype(np.float32)
            continue
        except:
            pass
            
    return df[sorted(df.columns)].sort_values(by=['ticker', 'period'])

In [9]:
df_preproc = preprocess(df)

In [12]:
nasdaq = pd.read_csv('nasdaq_smpl_prices.csv', sep=';')
print(nasdaq.shape)
nasdaq.head()

(13085, 1481)


,Index,AAL,AAN,AAON,AAP,AATC,ABC,ABIO,ABNB,ABOS,...,ZD,ZGNX,ZION,ZM,ZNGA,ZNTE,ZSAN,ZVO,ZWRK,ZYNE
0,1970-01-02,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,1970-01-05,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,1970-01-06,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,1970-01-07,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,1970-01-08,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [22]:
nasdaq_df = pd.melt(nasdaq, id_vars=['Index'])
nasdaq_df.columns = ['date', 'ticker', 'price']
nasdaq_df['date'] = pd.to_datetime(nasdaq_df['date'], format='%Y-%m-%d')
print(nasdaq_df.shape)
nasdaq_df.head()

(19365800, 3)


,date,ticker,price
0,1970-01-02,AAL,NaN
1,1970-01-05,AAL,NaN
2,1970-01-06,AAL,NaN
3,1970-01-07,AAL,NaN
4,1970-01-08,AAL,NaN


In [35]:
nasdaq_df['date_month'] = nasdaq_df['date'].dt.strftime("%Y-%m")

In [36]:
nasdaq_df_month = nasdaq_df.groupby(by=['ticker', 'date_month'])['price'].mean().reset_index()

In [42]:
nasdaq_df_month = nasdaq_df_month.sort_values(by=['ticker', 'date_month']).reset_index(drop=True)

In [73]:
lag_prices = [
    "lag_price_1Q",
    "lag_price_1H",
    "lag_price_3Q",
    "lag_price_1Y",
    "lag_price_3H",
    "lag_price_2Y",
    "lag_price_3Y",
    "lag_price_5Y"
]
lag_months = [3, 6, 9, 12, 18, 24, 36, 60]

for lag_price, lag_month in zip(lag_prices, lag_months):
    nasdaq_df_month[lag_price] = nasdaq_df_month.groupby('ticker')['price'].shift(periods=lag_month)

In [116]:
stat_months = [3, 6, 9, 12, 18, 24, 36, 60]

for stat_month in stat_months:
    nasdaq_df_month[f"price_MA_{stat_month}"] = nasdaq_df_month.groupby('ticker')['price'].rolling(stat_month, min_periods=1, closed='left').mean().values

In [74]:
lead_prices = [
    "lead_price_1Y",
    "lead_price_2Y",
    "lead_price_3Y"
]
lead_months = [-12, -24, -36]

for lead_price, lead_month in zip(lead_prices, lead_months):
    nasdaq_df_month[lead_price] = nasdaq_df_month.groupby('ticker')['price'].shift(periods=lead_month)

In [88]:
stat_prices = [
    "min_5Y_price",
    "max_5Y_price",
    "min_3Y_price",
    "max_3Y_price"
]
stat_months = [60, 60, 36, 36]
stat_funcs = ['min', 'max', 'min', 'max']

for stat_price, stat_month, stat_func in zip(stat_prices, stat_months, stat_funcs):
    nasdaq_df_month[stat_price] = nasdaq_df_month.groupby('ticker')['price'].rolling(stat_month, min_periods=1, closed='left').agg([stat_func]).values

In [90]:
lead_prices = [
    "lead_price_1Y",
    "lead_price_2Y",
    "lead_price_3Y"
]
target_prices = [
    "Target_1Y",
    "Target_2Y",
    "Target_3Y"
]

for lead_price, target_price in zip(lead_prices, target_prices):
    nasdaq_df_month[target_price] = nasdaq_df_month[lead_price] / nasdaq_df_month['price'] - 1

In [118]:
df_res = pd.merge(df_preproc, nasdaq_df_month, left_on=['ticker', 'Fiscal Year'], right_on=['ticker', 'date_month'], how='left')

In [120]:
tonum = df_preproc.columns[df_preproc.dtypes != 'object'].tolist()
tonum.remove('period')

In [121]:
lag_years = range(1, 11)

for feat in tqdm(tonum):
    for lag_year in lag_years:
        df_res[f"{feat}_lag_{lag_year}Y"] = df_res.groupby('ticker')[feat].shift(periods=lag_year)

  4%|▎         | 9/253 [00:00<00:06, 37.93it/s]/var/folders/rt/1tkqxcb94vs7knvpqxqg_k100000gp/T/ipykernel_4470/2414555379.py:5: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_res[f"{feat}_lag_{lag_year}Y"] = df_res.groupby('ticker')[feat].shift(periods=lag_year)
100%|██████████| 253/253 [00:06<00:00, 39.02it/s]


In [124]:
df_res['Debt2Cash'] = (df_res['Long-Term Debt'] + df_res['Short-Term Debt']) / df_res['Cash And Cash Equivalents']
df_res['ShortDebt2Cash'] = df_res['Short-Term Debt'] / df_res['Cash And Cash Equivalents']
df_res['LongDebt2Cash'] = df_res['Long-Term Debt'] / df_res['Cash And Cash Equivalents']
df_res['OperIncome2Debt'] = df_res['Operating Income'] / (df_res['Long-Term Debt'] + df_res['Short-Term Debt'])
df_res['OperIncome2ShortDebt'] = df_res['Operating Income'] / df_res['Short-Term Debt']
df_res['OperIncome2LongDebt'] = df_res['Operating Income'] / df_res['Long-Term Debt']

/var/folders/rt/1tkqxcb94vs7knvpqxqg_k100000gp/T/ipykernel_4470/3316125249.py:1: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_res['Debt2Cash'] = (df_res['Long-Term Debt'] + df_res['Short-Term Debt']) / df_res['Cash And Cash Equivalents']
/var/folders/rt/1tkqxcb94vs7knvpqxqg_k100000gp/T/ipykernel_4470/3316125249.py:2: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_res['ShortDebt2Cash'] = df_res['Short-Term Debt'] / df_res['Cash And Cash Equivalents']
/var/folders/rt/1tkqxcb94vs7knvpqxqg_k100000gp/T/ipykernel_4470/33161252

In [125]:
feats = [
    "Debt2Cash","ShortDebt2Cash","LongDebt2Cash",
    "OperIncome2Debt","OperIncome2ShortDebt","OperIncome2LongDebt"
]

lag_years = range(1, 11)

for feat in tqdm(feats):
    for lag_year in lag_years:
        df_res[f"{feat}_lag_{lag_year}Y"] = df_res.groupby('ticker')[feat].shift(periods=lag_year)

  0%|          | 0/6 [00:00<?, ?it/s]/var/folders/rt/1tkqxcb94vs7knvpqxqg_k100000gp/T/ipykernel_4470/59200738.py:10: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_res[f"{feat}_lag_{lag_year}Y"] = df_res.groupby('ticker')[feat].shift(periods=lag_year)
100%|██████████| 6/6 [00:00<00:00, 33.46it/s]


In [126]:
feats = [
    "lag_price_1Q",
    "lag_price_1H",
    "lag_price_3Q",
    "lag_price_1Y",
    "lag_price_3H",
    "lag_price_2Y",
    "lag_price_3Y",
    "lag_price_5Y",
    "min_5Y_price",
    "max_5Y_price",
    "min_3Y_price",
    "max_3Y_price",
    "price_MA_3",
    "price_MA_6",
    "price_MA_9",
    "price_MA_12",
    "price_MA_24",
    "price_MA_36",
    "price_MA_60"
]
for feat in feats:
    df_res[f'p2{feat}'] = df_res['price'] / df_res[feat]

/var/folders/rt/1tkqxcb94vs7knvpqxqg_k100000gp/T/ipykernel_4470/1925831846.py:23: PerformanceWarning: DataFrame is highly fragmented.  This is usually the result of calling `frame.insert` many times, which has poor performance.  Consider joining all columns at once using pd.concat(axis=1) instead.  To get a de-fragmented frame, use `newframe = frame.copy()`
  df_res[f'p2{feat}'] = df_res['price'] / df_res[feat]


In [130]:
df_res.shape

(23335, 2909)

In [128]:
df_res.to_csv('Annual_Fin_Table_Target.csv', index=False)